In [ ]:
import manganite
%load_ext manganite

# Supplier Sourcing

We will now formulate and solve a supplier sourcing problem.

New Bedford Steel (NBS) procures coking coal to produce steel. For next year’s
production, NBS has solicited the following bids from eight different coal mines:

<pre>
$$
\begin{array}{c | cccccccc}
\hline
\hline
 & Ashley & Bedfort & Consol &Dunby &Earlam &Florence &Gaston &Hopt \\
\hline
Price (\$/mt) & 49,500 & 50,000 & 61,000 &63,500 &66,500 &71,000 &72,500 &80,000  \\
Union? & yes & yes & no & yes &no &yes &no &no \\
Transport & rail & truck & rail &truck &truck &truck &rail &rail \\
Volatility (percentage) & 15 & 16 & 18 & 20 &21 &22 &23 &25 \\
Capacity (mt/yr) &300 &600 &510 &655 &575 &680 &450 &490 \\
\hline
\hline
\end{array}
$$
</pre>

NBS wants to procure 1,225mt of coking coal with an average volatility of at least 19%. To avert adverse labour relations, at least 50% of the coal should come from union mines. Moreover, at most 650mt (720mt) can be transported via rail (trucks).

*   How much should NBS procure from each mine so as to minimize total costs?
*   What are the total costs? What are the average costs per mt?

In [ ]:
import cvxpy as cp
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
%%mnn widget --type table --var supplier_df --tab Inputs --header Suppliers
supplier_df = pd.read_excel('SupplierSourcing.xlsx',index_col=0)

In [ ]:
%%mnn widget --type plot --var cap_chart --tab Inputs --header Capacities
cap_chart = px.bar(
    data_frame = (supplier_df
                  .reset_index()
                  .melt(id_vars="index")
                  .loc[lambda x:x["index"]=="Capacity (mt/yr)"]),
    x="variable", 
    y="value", 
    labels={"variable":"Supplier","value":"Capacity"}, 
    template="plotly_white"
)

In [ ]:
supplierdata = supplier_df.to_numpy() #dataframe to numpy array
n = len(supplierdata[0]) #number of suppliers
p = (supplierdata[0,:]) #prices
u = (supplierdata[1,:]) #whether from union
t = (supplierdata[2,:]) #whether rail transportation
v = (supplierdata[3,:]) #volatility
c = (supplierdata[4,:]) #capacities

x = cp.Variable(n)

In [ ]:
%%mnn widget --type slider 0:2000:1 --var sum --tab Inputs --position -1 0 6 --header Sum
sum = 1225

In [ ]:
%%mnn execute --on button "Optimize" --returns x
objective = cp.Minimize(p@x)

constraints = [cp.sum(x) == sum]
constraints.append(v@x >= (0.19)*cp.sum(x))
constraints.append(u@x >= (1/2)*cp.sum(x))
constraints.append(t@x <= 650)
constraints.append((1-t)@x <= 720)
constraints.append(x[:] <= c[:])
constraints.append(x >= 0)

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.GUROBI)

In [ ]:
%%mnn widget --type plot --var sol_chart --tab Results --header Solution
sol_chart = px.bar(data_frame=pd.DataFrame(
    {"Suppliers":supplier_df.columns,
     "Order volume": x.value}),
       x="Suppliers",
       y="Order volume",
       template="plotly_white")